In [3]:
import pandas as pd

# Load the dataset from an Excel file
file_path = 'Movie_Dataset1.xlsx'
df = pd.read_excel(file_path)
print(df)

                    Movie   Year  Certificate             Genre  \
0         Aasha Encounter  2022.0           A    Thriller,Drama   
1              Induvadana  2022.0          UA   Horror, Romance   
2                    1945  2022.0          UA        War,Action   
3       Atithi Devo Bhava  2022.0          UA    Romance, Drama   
4              Bangarraju  2022.0          UA      Action,Drama   
...                   ...     ...         ...               ...   
1594     Shyam Singha Roy  2021.0          UA  Thriller,Romance   
1595  WWW : Who Where Why  2021.0          UA          Thriller   
1596           Gudaputani  2021.0          UA            Comedy   
1597      Arjuna Phalguna  2021.0          UA   Comedy,Thriller   
1598             Senapati  2021.0          UA    Thriller,Drama   

                                               Overview  Runtime  Rating  \
0     Disha gang assault and killing on 27 November ...       99     6.9   
1     Vasu is a forest officer who falls in

In [24]:
# Initialize the emotion analysis pipeline
emotion_classifier = pipeline('text-classification', model='j-hartmann/emotion-english-distilroberta-base', top_k=None)

# Function to get the dominant emotions for a combined text
def get_dominant_emotions(text):
    predictions = emotion_classifier(text)[0]  # Extract the inner list directly
    emotions = [pred['label'] for pred in predictions if pred['score'] > 0.5]  # Filter emotions with a high score
    return emotions if emotions else [predictions[0]['label']]
    # Convert predictions to a list of dictionaries
    # predictions = [dict(pred) for pred in predictions]
    # print(predictions)
    # # Check if the 'score' key is present in the predictions
    # if all(pred.get('score') is not None for pred in predictions):
    #     emotions = [pred['label'] for pred in predictions if pred['score'] > 0.5]
    # else:
    #     # If 'score' is not present, use the first prediction as the dominant emotion
    #     emotions = [predictions[0]['label']]

    # return emotions

# Combine Genre and Overview for emotion analysis
df['Combined'] = df['Genre'].astype(str) + ' ' + df['Overview'].astype(str)

# Debugging: Print structure of predictions for a sample text
sample_text = df['Combined'].iloc[0]
sample_predictions = emotion_classifier(sample_text)
print(sample_predictions)

# Apply the function to the Combined column to create the Emotions column
df['Emotions'] = df['Combined'].apply(lambda x: get_dominant_emotions(x))

print(df['Emotions'])

[[{'label': 'anger', 'score': 0.3540501296520233}, {'label': 'surprise', 'score': 0.25246721506118774}, {'label': 'fear', 'score': 0.2474737912416458}, {'label': 'disgust', 'score': 0.12261030077934265}, {'label': 'sadness', 'score': 0.012052446603775024}, {'label': 'neutral', 'score': 0.007563478779047728}, {'label': 'joy', 'score': 0.0037826274055987597}]]
0         [anger]
1          [fear]
2       [sadness]
3       [neutral]
4       [sadness]
          ...    
1594        [joy]
1595       [fear]
1596       [fear]
1597        [joy]
1598    [sadness]
Name: Emotions, Length: 1599, dtype: object


In [25]:
for value in df['Emotions']:
    print(value[0])

anger
fear
sadness
neutral
sadness
disgust
neutral
sadness
fear
fear
sadness
anger
anger
fear
disgust
sadness
sadness
neutral
anger
fear
joy
joy
fear
joy
sadness
joy
anger
anger
neutral
fear
sadness
anger
sadness
anger
fear
joy
sadness
fear
joy
sadness
fear
sadness
joy
sadness
sadness
joy
anger
sadness
neutral
anger
sadness
joy
fear
anger
fear
joy
fear
disgust
neutral
sadness
anger
sadness
fear
disgust
sadness
fear
fear
fear
anger
fear
fear
joy
joy
joy
sadness
fear
fear
sadness
fear
disgust
joy
anger
anger
neutral
fear
sadness
neutral
fear
joy
joy
anger
joy
joy
fear
sadness
joy
sadness
fear
fear
joy
sadness
fear
fear
joy
anger
anger
fear
fear
sadness
joy
joy
sadness
anger
sadness
joy
joy
sadness
fear
joy
joy
joy
anger
joy
joy
joy
joy
joy
joy
joy
fear
sadness
anger
joy
anger
sadness
joy
fear
anger
fear
anger
joy
sadness
sadness
anger
joy
joy
joy
anger
joy
fear
anger
joy
sadness
joy
joy
anger
anger
joy
joy
anger
joy
sadness
joy
joy
anger
sadness
joy
fear
joy
joy
joy
sadness
joy
anger
joy

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

# Vectorize the 'Combined' column
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['Combined'])

# Binarize the emotions
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['Emotions'])
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = OneVsRestClassifier(LogisticRegression())
model.fit(X_train, y_train)
def recommend_movies(emotion):
    emotion_vector = mlb.transform([[emotion]])
    scores = model.predict_proba(X)[:, emotion_vector.argmax()]
    df['Score'] = scores
    recommendations = df.sort_values(by='Score', ascending=False).head(5)
    return recommendations[['Movie', 'Genre', 'Overview', 'Score']]

# Example usage
recommended_movies = recommend_movies('fear')
print(recommended_movies)

                  Movie                                  Genre  \
696             Anasuya   Crime, Mystery, Thriller               
1056          Ice Cream           Horror, Thriller               
995   Case No. 666/2013           Horror, Thriller               
227           Arundhati  Horror, Mystery, Thriller               
487                Dora   Action, Horror, Thriller               

                                               Overview     Score  
696       A spine-chilling suspense thriller about a...  0.583634  
1056      The protagonist is obsessed with eating ic...  0.579668  
995       Three buddies Bhaskar, Chaitanya and Durga...  0.550687  
227       A brave queen battles an evil mystic. Thre...  0.530159  
487       A father and a daughter purchase a used ca...  0.530083  


In [43]:
import pickle

In [49]:
model = 'MovieRecommendations.ipynb'
with open('movie_recommend_working_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Close the file
f.close()